In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
import numpy as np
import pickle

from common import Data, Split, Batches, load_data, encode_y, load_split
from utils import ProgressBar, evaluate
from pack import Pack
from model import Model, TextOnlyModel
from torch_models import MultiLayerFCReLUClassifier, Encoder2
from cls import BRClassifier
from train import train_epoches

%load_ext autoreload
%autoreload 2

In [2]:
genre_list = pickle.load(open("../data/tmdb_genres_list.pkl", 'rb'))

GENRES = load_data("../local/genres.pkl")
train = load_split("../local/train.pkl")
val = load_split("../local/val.pkl")
test = load_split("../local/test.pkl")
embedding = torch.load('../local/embedding.pth')
OVERVIEWS_ENCODED = load_data("../local/overviews_encoded.pkl")
TITLES_ENCODED = load_data("../local/titles_encoded.pkl")

In [3]:
classifier = BRClassifier(dims=[1024, 512], num_class=19, encoding_size=1024, cuda=True)
encoder = Encoder2(encoder=torch.nn.LSTM, embedding=embedding, input_channel=embedding.embedding_dim,
                  hidden_dim=512, num_layers=3, cuda=True, bidirectional=True, dropout=0)
model = TextOnlyModel(encoder, classifier, OVERVIEWS_ENCODED, GENRES)
loss = torch.nn.BCEWithLogitsLoss().cuda()

In [4]:
adam = torch.optim.Adam(filter(lambda p:p.requires_grad, model.parameters()))

optimizer = adam
scheduler = None

In [5]:
loss_hist = []
n_epochs = 10
for i in range(0,20):
    epoch_losses = train_epoches(n_epochs=n_epochs, model=model, train=train, loss=loss, val=val,
                  batch_size=32, optimizer=optimizer, scheduler=scheduler)
    loss_hist.append(epoch_losses)
    bn = (i+1)*n_epochs
    torch.save(model.encoder, "saved/overview-lstm2/encoder_{}_{}.pth".format(bn, str(epoch_losses[1][1][-1])[:4]))
    torch.save(model.classifier, "saved/overview-lstm2/cls_{}_{}.pth".format(bn, str(epoch_losses[1][1][-1])[:4]))

......................................5.....................................10......................................15.....................................20......................................25.....................................30......................................35.....................................40......................................45.....................................50......................................55.....................................60......................................65.....................................70......................................75.....................................80......................................85.....................................90......................................95.....................................100 F
epoch 0: 0.2301148
Val:	P(mi) 0.0 	R(mi): 0.0 	F1(mi): 0.0
P(ma) 0.0 	R(ma): 0.0 	F1(ma): 0.0
P(w) 0 	R(w): 0 	F1(w): 0


.

/home/hongxiangqiu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/home/hongxiangqiu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/home/hongxiangqiu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/hongxiangqiu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


.....................................5.....................................10......................................15.....................................20......................................25.....................................30......................................35.....................................40......................................45.....................................50......................................55.....................................60......................................65.....................................70......................................75.....................................80......................................85.....................................90......................................95.....................................100 F
epoch 1: 0.2256135
Val:	P(mi) 0.0 	R(mi): 0.0 	F1(mi): 0.0
P(ma) 0.0 	R(ma): 0.0 	F1(ma): 0.0
P(w) 0 	R(w): 0 	F1(w): 0


......................................5.....................................10........

......................................5.....................................10......................................15.....................................20......................................25.....................................30......................................35.....................................40......................................45.....................................50......................................55.....................................60......................................65.....................................70......................................75.....................................80......................................85.....................................90......................................95.....................................100 F
epoch 9: 0.13446751
Val:	P(mi) 0.355224609375 	R(mi): 0.5517633674630261 	F1(mi): 0.4321996138422694
P(ma) 0.24829782222821242 	R(ma): 0.35273280459989415 	F1(ma): 0.2813262082764469
P(w) 0.43400543566191613

Val:	P(mi) 0.47216796875 	R(mi): 0.550997150997151 	F1(mi): 0.5085458848277675
P(ma) 0.3868311490316593 	R(ma): 0.5142204354550869 	F1(ma): 0.43462547957315556
P(w) 0.4924885948993364 	R(w): 0.550997150997151 	F1(w): 0.5164211834512606


......................................5.....................................10......................................15.....................................20......................................25.....................................30......................................35.....................................40......................................45.....................................50......................................55.....................................60......................................65.....................................70......................................75.....................................80......................................85.....................................90......................................95............

...

......................................5.....................................10......................................15.....................................20......................................25.....................................30......................................35.....................................40......................................45.....................................50......................................55.....................................60......................................65.....................................70......................................75.....................................80......................................85.....................................90......................................95.....................................100 F
epoch 2: 0.0036232325
Val:	P(mi) 0.4873046875 	R(mi): 0.5709382151029748 	F1(mi): 0.5258166491043204
P(ma) 0.4085601691607773 	R(ma): 0.5893660302239354 	F1(ma): 0.4711341837297902
P(w) 0.5101125161341326 	R

Val:	P(mi) 0.511474609375 	R(mi): 0.5557029177718833 	F1(mi): 0.5326722603610475
P(ma) 0.4298116825810588 	R(ma): 0.5348310037492194 	F1(ma): 0.47040663206109334
P(w) 0.5294396649694694 	R(w): 0.5557029177718833 	F1(w): 0.537964379115973


......................................5.....................................10......................................15.....................................20......................................25.....................................30......................................35.....................................40......................................45.....................................50......................................55.....................................60......................................65.....................................70......................................75.....................................80......................................85.....................................90......................................95..........

......................................5.....................................10......................................15.....................................20......................................25.....................................30......................................35.....................................40......................................45.....................................50......................................55.....................................60......................................65.....................................70......................................75.....................................80......................................85.....................................90......................................95.....................................100 F
epoch 7: 0.002796664
Val:	P(mi) 0.503173828125 	R(mi): 0.5580828594638505 	F1(mi): 0.5292078572345615
P(ma) 0.4246052951848302 	R(ma): 0.562260536982586 	F1(ma): 0.47529454367195656
P(w) 0.5233684575367241 	

Val:	P(mi) 0.484375 	R(mi): 0.5804564072557051 	F1(mi): 0.5280809156241683
P(ma) 0.41908238142744453 	R(ma): 0.5689107037331875 	F1(ma): 0.4756775770317976
P(w) 0.5038615179488082 	R(w): 0.5804564072557051 	F1(w): 0.5350329824032334


......................................5.....................................10......................................15.....................................20......................................25.....................................30......................................35.....................................40......................................45.....................................50......................................55.....................................60......................................65.....................................70......................................75.....................................80......................................85.....................................90......................................95...............

KeyboardInterrupt: 

In [ ]:
evaluate(test, model, 128)

In [ ]:
torch.save(optimizer, "saved/overview-lstm2/optimizer.pth")